In [1]:
import tensorflow as tf
import tensorflow.feature_column as fc 

import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from random import shuffle
 
os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)


In [2]:

data_input = ['{0:020b}'.format(i) for i in range(2**15)]
shuffle(data_input)
data_input = [map(int,i) for i in data_input]
ti  = []
for i in data_input:    
    temp_list = []
    for j in i:
        temp_list.append([j])
#         print(j)
    ti.append(np.array(temp_list))
data_input = ti



In [3]:
data_output = []
 
for i in data_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*21)
    temp_list[count]=1
    data_output.append(temp_list)
    
data_input = np.array(data_input)
data_output = np.array(data_output)
print("i.shape:", data_input.shape, "o.shape:", data_output.shape)

i.shape: (32768, 20, 1) o.shape: (32768, 21)


In [4]:
NUM_EXAMPLES = 10000
test_input = data_input[NUM_EXAMPLES:]
test_output = data_output[NUM_EXAMPLES:] #everything beyond 10,000
 
train_input = data_input[:NUM_EXAMPLES]
train_output = data_output[:NUM_EXAMPLES] #till 10,000
print(test_input.shape)
print(test_output.shape)

print(train_input.shape)
print(train_output.shape)


(22768, 20, 1)
(22768, 21)
(10000, 20, 1)
(10000, 21)


In [5]:
num_layers = 3
num_hidden = 24
    
tf.reset_default_graph()

with tf.device('/gpu:0'):
    data = tf.placeholder(tf.float32, [None, 20,1])
    target = tf.placeholder(tf.float32, [None, 21])

    lstm = tf.contrib.cudnn_rnn.CudnnLSTM(3, num_hidden)
    val, _  = lstm(data)
    val = tf.transpose(val, [1, 0, 2])
    t = tf.shape(val)
    last = tf.gather(val,(t[0] - 1))

    weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
    bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))

    prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

    entropy = target * tf.log(tf.clip_by_value(prediction,1e-10,1.0))
    cross_entropy = -tf.reduce_sum(entropy)
    optimizer = tf.train.AdamOptimizer()
    minimize = optimizer.minimize(cross_entropy)
    
with tf.device('/cpu:0'):
    mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
    error = tf.reduce_mean(tf.cast(mistakes, tf.float32))


# init_op = tf.initialize_all_variables()



/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [6]:
num_epoch = 50
batch_size = 512
no_of_batches = int(len(train_input)/batch_size)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(num_epoch):
        p = 0
        for batch in range(no_of_batches):
            p1 = p
            p = p + batch_size
            x, y = train_input[p1:p], train_output[p1:p]
            x_val, y_val = test_input, test_output
            sess.run(minimize,{data: x, target: y})
            train_error = sess.run(error, {data:x, target:y})
            val_error = sess.run(error, {data:x_val, target:y_val})
            
            print("Epoch:", epoch, "Batch:", batch, "Training Error:", train_error, "Validation Error:", val_error)



Epoch: 0 Batch: 0 Training Error: 0.94921875 Validation Error: 0.9410576
Epoch: 0 Batch: 1 Training Error: 0.8105469 Validation Error: 0.8042428
Epoch: 0 Batch: 2 Training Error: 0.8300781 Validation Error: 0.8039793
Epoch: 0 Batch: 3 Training Error: 0.765625 Validation Error: 0.8042867
Epoch: 0 Batch: 4 Training Error: 0.796875 Validation Error: 0.8049016
Epoch: 0 Batch: 5 Training Error: 0.80859375 Validation Error: 0.8052969
Epoch: 0 Batch: 6 Training Error: 0.7910156 Validation Error: 0.8048138
Epoch: 0 Batch: 7 Training Error: 0.8125 Validation Error: 0.8045063
Epoch: 0 Batch: 8 Training Error: 0.7910156 Validation Error: 0.8042867
Epoch: 0 Batch: 9 Training Error: 0.80859375 Validation Error: 0.804111
Epoch: 0 Batch: 10 Training Error: 0.80859375 Validation Error: 0.8039793
Epoch: 0 Batch: 11 Training Error: 0.8339844 Validation Error: 0.8039793
Epoch: 0 Batch: 12 Training Error: 0.80078125 Validation Error: 0.80375963
Epoch: 0 Batch: 13 Training Error: 0.8125 Validation Error: 0

KeyboardInterrupt: 

In [ ]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

# num_epoch = 50
# batch_size = 64
# no_of_batches = int(len(train_input)/batch_size)

# for i in range(num_epoch):
#     ptr = 0
#     for j in range(no_of_batches):
#         inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
#         ptr+=batch_size
#         sess.run(minimize,{data: inp, target: out})
#     print("Epoch - ",str(i))
# incorrect = sess.run(error,{data: test_input, target: test_output})
# print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
# sess.close()



In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    a = sess.run(prediction,{data: [[[1],[0],[0],[1],[1],[0],[1],[1],[1],[0],[1],[0],[0],[1],[1],[0],[1],[1],[1],[0]]]})
    print(a)
    print(a.shape)
sess.close()